In [1]:
import pickle
import numpy as np
from sklearn.preprocessing import RobustScaler,MinMaxScaler
X_ppg,X_qual,X_acc,y_rr,y_respiration,y_inspiration,y_expiration,groups,X_respiration,y_activity,y_label = pickle.load(open('../../affect-recognition/data/tabular_data_8.p','rb'))

In [ ]:
from sklearn.preprocessing import OneHotEncoder
index = np.where((y_activity!=0)&(y_activity!=3)&(y_activity!=6)&(y_activity!=8)&(y_activity!=2))[0]
X_acc = X_acc[index]
y_activity = y_activity[index]
groups = groups[index]
y_activity = OneHotEncoder().fit_transform(y_activity.reshape(-1,1)).todense()
X_fft = np.fft.fft(X_acc,axis=1)
X_fft_real = X_fft.real
X_fft_imag = X_fft.imag
X_fft = np.concatenate([X_fft_real,X_fft_imag],axis=2)
max_value = X_fft.max()
X_fft = X_fft/max_value
X_fft.shape

In [ ]:
# example of training an conditional gan on the fashion mnist dataset
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input,GRU,Bidirectional
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D,Conv1D,MaxPool1D,BatchNormalization
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Concatenate
import warnings
warnings.filterwarnings('ignore')

# define the standalone discriminator model
def define_discriminator(in_shape=(256,6), n_classes=4):
    # label input
    in_label = Input(shape=(n_classes,))
    # embedding for categorical input
    n_nodes = in_shape[0] * in_shape[1]
#     li = Dense(20)(in_label)
#     li = Embedding(n_classes, 100)(in_label)
    # scale up to image dimensions with linear activation
    li = Dense(n_nodes)(in_label)
#     # reshape to additional channel
    li = Reshape((in_shape[0], in_shape[1]))(li)
    # image input
    in_image = Input(shape=in_shape)
#     # concat label as a channel
    merge = Concatenate()([in_image, li])
#     # downsample
    fe = Conv1D(100,4, padding='same',activation='tanh')(merge)
#     fe = LeakyReLU(alpha=0.2)(fe)
    fe = MaxPool1D(2)(fe)
#     fe = BatchNormalization()(fe)
#     fe = Dropout(0.2)(fe)
    # downsample
    fe = Conv1D(100,8,padding='same',activation='tanh')(fe)
#     fe = LeakyReLU(alpha=0.2)(fe)
    fe = MaxPool1D(4)(fe)
    
    fe = Conv1D(100,8,padding='same',activation='tanh')(fe)
#     fe = LeakyReLU(alpha=0.2)(fe)
    fe = MaxPool1D(8)(fe)
    
#     fe = BatchNormalization()(fe)
#     fe = Dropout(0.4)(fe)
    # flatten feature maps
    fe = Flatten()(fe)
    # dropout
    fe = Dropout(0.4)(fe)
    # output
    fe  = Dense(10, activation='sigmoid')(fe)
    out_layer = Dense(1, activation='sigmoid')(fe)
    # define model
    model = Model([in_image,in_label], out_layer)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# define the standalone generator model
def define_generator(latent_dim, n_classes=4,out_shape=(256,6)):
    # label input
    in_label = Input(shape=(n_classes,))
    
    # embedding for categorical input
    li = Dense(out_shape[0]*n_classes//4,activation='relu')(in_label)
    li = Reshape((out_shape[0],n_classes//4))(li)
    li = Conv1D(256,10, padding='same',activation='relu')(li)
#     li = Flatten()(li)
#     li = Dense(out_shape[0]*n_classes//2)(li)
#     li = Dense(out_shape[0]*n_classes)(li)
#     li = Reshape((out_shape[0],n_classes))(li)
    in_lat = Input(shape=(latent_dim,))
    gen = Dense(out_shape[0]*latent_dim//10,activation='tanh')(in_lat)
    gen = Reshape((out_shape[0],latent_dim//10))(gen)
    gen = Conv1D(256,10, padding='same',activation='tanh')(gen)
    merge = Concatenate()([gen, li])
    fe = Conv1D(100,10, padding='same',activation='tanh')(merge)
    fe = Conv1D(100,5, padding='same',activation='tanh')(fe)
    fe = Conv1D(6,5, padding='same',activation='tanh')(fe)
    
#     fe = BatchNormalization()(fe)
#     fe = Dropout(0.2)(fe)
    # downsample
#     fe = Conv1D(100,8,padding='same',activation='tanh')(fe)
# #     fe = LeakyReLU(alpha=0.2)(fe)
#     fe = MaxPool1D(4)(fe)
    
#     fe = Conv1D(100,8,padding='same',activation='tanh')(fe)
# #     fe = LeakyReLU(alpha=0.2)(fe)
#     fe = MaxPool1D(8)(fe)
#     # foundation for 7x7 image
#     n_nodes = 1 * 1 
#     gen = Dense(n_nodes)(in_lat)
#     gen = LeakyReLU(alpha=0.2)(gen)
#     gen = Reshape((1, 1))(gen)
#     # merge image gen and label input
#     merge = Concatenate()([gen, li])
#     # upsample to 14x14
#     gen = Conv1D(8, 1, padding='same',activation='relu')(merge)
# #     gen = LeakyReLU(alpha=0.2)(gen)
#     gen = Conv1D(16, 1, padding='same',activation='relu')(gen)
# #     gen = LeakyReLU(alpha=0.2)(gen)
#     gen = Conv1D(32, 1, padding='same',activation='relu')(gen)
# #     gen = LeakyReLU(alpha=0.2)(gen)
#     gen = Conv1D(64, 1, padding='same',activation='relu')(gen)
# #     gen = LeakyReLU(alpha=0.2)(gen)
#     gen = Conv1D(128, 1, padding='same',activation='relu')(gen)
# #     gen = LeakyReLU(alpha=0.2)(gen)
#     gen = Conv1D(256, 1, padding='same',activation='sigmoid')(gen)
# #     gen = LeakyReLU(alpha=0.2)(gen)
#     out_layer = Reshape((256,1))(gen)
# #     out_layer = Bidirectional(GRU(10,return_sequences=True,activation='sigmoid'))(out_layer)
# #     out_layer = Dense(100,activation='linear')(out_layer)
# #     out_layer = Dense(1,activation='linear')(out_layer)
# #     # upsample to 28x28
# #     gen = Conv2DTranspose(256, (4,4), strides=(2,2), padding='same')(gen)
# #     gen = LeakyReLU(alpha=0.2)(gen)
# #     # output
# #     out_layer = Conv2D(1, (7,7), activation='tanh', padding='same')(gen)
#     # define model
    model = Model([in_lat,in_label], fe)
#     model.compile(loss='mse', optimizer='adam')
#     model = Model([in_label,in_lat],[fe])
    return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    
    # get noise and label inputs from generator model
    gen_noise, gen_label = g_model.input
    # get image output from the generator model
    gen_output = g_model(g_model.input)
    # connect image output and label input from generator as inputs to discriminator
    gan_output = d_model([gen_output, gen_label])
    d_model.trainable = False
    # define gan model as taking noise and label and outputting a classification
    model = Model([gen_noise, gen_label], gan_output)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

%matplotlib inline
# load fashion mnist images
def load_real_samples():
    # load dataset
#     (trainX, trainy), (_, _) = load_data()
    
    # expand to 3d, e.g. add channels
#     X = expand_dims(trainX, axis=-1)
#     # convert from ints to floats
#     X = X.astype('float32')
#     # scale from [0,255] to [-1,1]
#     X = (X - 127.5) / 127.5
    return [X_fft, y_activity]

activity_label = {0:'Sitting',1:'Cycling',2:'Driving',3:'Walking'}

# # select real samples
def generate_real_samples(dataset, n_samples):
    # split into images and labels
    images, labels = dataset
    # choose random instances
    ix = randint(0, images.shape[0], n_samples)
    # select images and labels
    X, labels = images[ix], labels[ix]
#     first_row = X[0]*max_value
#     aclx = first_row[:,0] + 1j*first_row[:,3]
#     acly = first_row[:,1] + 1j*first_row[:,4]
#     aclz = first_row[:,2] + 1j*first_row[:,5]
# #     print(first_row.shape)
#     label = labels[0].reshape(-1)
#     plt.figure()
#     plt.plot(np.fft.ifft(aclx))
#     plt.plot(np.fft.ifft(acly))
#     plt.plot(np.fft.ifft(aclz))
#     plt.title(activity_label[np.argmax(label)])
#     plt.ylabel('real')
#     plt.show()
    # generate class labels
    y = ones((n_samples, 1))
    return [X, labels], y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=4):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    z_input = x_input.reshape(n_samples, latent_dim)
    # generate labels
    labels = randint(0, n_classes, n_samples)
    labels = OneHotEncoder().fit_transform(labels.reshape(-1,1)).todense()
#     print(labels.shape,labels[0])
    return [z_input, labels]

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    z_input, labels_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    images = generator.predict([z_input, labels_input])
    first_row = images[0]*max_value
    aclx = first_row[:,0] + 1j*first_row[:,3]
    acly = first_row[:,1] + 1j*first_row[:,4]
    aclz = first_row[:,2] + 1j*first_row[:,5]
#     print(first_row.shape)
    label = labels_input[0].reshape(-1)
    plt.figure()
    plt.plot(np.fft.ifft(aclx))
    plt.plot(np.fft.ifft(acly))
    plt.plot(np.fft.ifft(aclz))
    plt.title(activity_label[np.argmax(label)])
    plt.ylabel('Fake')
    plt.show()
#     plt.figure()
#     plt.plot(images[0])
#     plt.title(labels_input[0])
#     plt.ylabel('fake')
#     plt.show()
    # create class labels
    y = zeros((n_samples, 1))
    return [images, labels_input], y

# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=100):
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
#             print(X_real.shape,labels_real.shape,y_real.shape)
            # update discriminator model weights
            d_model.trainable = True
            d_loss1, dd1 = d_model.train_on_batch([X_real, labels_real], y_real)
#             # generate 'fake' examples
            [X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # update discriminator model weights
            d_model.trainable = True
            d_loss2, dd2 = d_model.train_on_batch([X_fake, labels], y_fake)
            print(dd1,dd2)
            # prepare points in latent space as input for the generator
            [z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator's error
            d_model.trainable = False
            g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)
            
            # summarize loss on this batch
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
                (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
        g_model.save('../../affect-recognition/data/models/cgan_activity_acl/'+str(i)+'_v1.h5')
        print('--'*80)
    # save the generator model
#     g_model.save('cgan_generator.h5')

# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()

d_model.summary()
# create the generator
g_model = define_generator(latent_dim)

g_model.summary()
# # # create the gan
gan_model = define_gan(g_model, d_model)

gan_model.summary()
# # # # load image data
dataset = load_real_samples()
# # # # train model
train(g_model, d_model, gan_model, dataset, latent_dim)

In [ ]:
np.unique(np.round(y_activity))

In [11]:
import matplotlib.pyplot as plt
import numpy as np
# plt.plot()
ppg1 = X_acc[100][:,0]
ppg1.shape

(256,)

In [19]:
np.fft.fftshift(np.fft.fft(ppg1))

array([-2.44140625e-03+0.00000000e+00j, -5.37856378e-04+1.40689605e-04j,
        7.19621680e-04+2.14367465e-04j,  1.00856828e-03-1.37057924e-04j,
        1.11022626e-03+1.17182606e-04j, -1.67006798e-03+1.37344349e-03j,
       -2.38220660e-03-2.41142556e-03j,  4.30368927e-04-7.01150087e-04j,
        4.08387115e-04+2.04109236e-04j,  8.34394982e-04-1.12707123e-03j,
       -7.76327724e-04-1.59300649e-03j, -1.98553233e-03-1.45688387e-03j,
       -7.29262507e-04+1.36846609e-03j, -4.77230998e-04+7.63658771e-04j,
        8.52698467e-04-2.71435715e-04j, -1.55793354e-03+1.27516650e-03j,
        7.83962538e-04+5.87876423e-04j,  2.00870254e-03-8.23927956e-04j,
       -1.13813979e-03+2.21536421e-04j, -9.29014523e-04+9.96496761e-05j,
        1.66116935e-04+6.91015173e-05j, -1.58171361e-04-4.54266250e-04j,
       -1.23346011e-03+2.21564854e-03j, -2.20530934e-03+1.64558397e-03j,
       -9.29649825e-04-1.25560106e-03j,  7.40049860e-04+9.36989076e-04j,
       -1.79514136e-03-6.71596949e-04j, -3.45914327

In [ ]:
plt.plot(np.fft.fftshift(np.fft.fftfreq(256, 1/64)),np.abs(np.fft.fft(ppg1-np.mean(ppg1),axis=0)))
plt.show()
plt.plot(np.fft.ifft(np.fft.fft(ppg1,axis=1),axis=1))
plt.show()
plt.plot(ppg1)
# np.abs(np.fft.fftshift(np.fft.fft(ppg1-np.mean(ppg1),axis=1))).shape

In [ ]:
from scipy.fft import dct, idct
import matplotlib.pyplot as plt
N = ppg1.shape[0]
t = np.linspace(0,20,N)
x = ppg1.reshape(-1)
y = dct(x, norm='ortho')
window = np.zeros(N)
window[:40] = 1
yr = idct(y*window, norm='ortho')
sum(abs(x-yr)**2) / sum(abs(x)**2)

plt.plot(t, x, '-bx')
plt.plot(t, yr, 'ro')
window = np.zeros(N)
window[:25] = 1
yr = idct(y*window, norm='ortho')
sum(abs(x-yr)**2) / sum(abs(x)**2)

plt.plot(t, yr, 'g+')
plt.legend(['x', '$x_{20}$', '$x_{15}$'])
plt.grid()
plt.show()

In [ ]:
plt.plot(y)

In [ ]:
from scipy.fft import fft, fftfreq, fftshift
# number of signal points
N = ppg1.shape[0]
# sample spacing
T = 1.0 / 64
x = np.linspace(0.0, N*T, N)
y = ppg1.reshape(-1)
yf = fft(y)
xf = fftfreq(N, T)
xf = fftshift(xf)
yplot = fftshift(yf)
import matplotlib.pyplot as plt
plt.plot(xf, (1.0/N) * np.abs(yplot))
plt.grid()
plt.show()

In [ ]:
plt.plot(xf,np.abs(yplot))

In [ ]:
# list(zip(xf,(1/N)*np.abs(yplot)))
 from scipy.stats import norm

In [ ]:
plt.hist(norm.rvs(size=100))

In [ ]:
np.random.normal(1,.1,50)

In [ ]:
np.fft.fft(ppg1,axis=1).real

In [ ]:
import tensorflow as tf
import keras.backend as K

In [ ]:
y = X_ppg[30]
yf = tf.signal.fft(tf.complex(y,np.zeros((len(y)))))
yr = tf.signal.ifft(yf/(10498*2))

In [ ]:
# plt.plot(y)
plt.plot(yr)

In [ ]:
tf.math.real(yf).numpy().max(),tf.math.real(yf).numpy().min()

In [ ]:
plt.hist(X_acc.reshape(-1))

In [ ]:
X_acc.reshape(-1).min()

In [ ]:
"""An implementation of the improved WGAN described in https://arxiv.org/abs/1704.00028

The improved WGAN has a term in the loss function which penalizes the network if its
gradient norm moves away from 1. This is included because the Earth Mover (EM) distance
used in WGANs is only easy to calculate for 1-Lipschitz functions (i.e. functions where
the gradient norm has a constant upper bound of 1).

The original WGAN paper enforced this by clipping weights to very small values
[-0.01, 0.01]. However, this drastically reduced network capacity. Penalizing the
gradient norm is more natural, but this requires second-order gradients. These are not
supported for some tensorflow ops (particularly MaxPool and AveragePool) in the current
release (1.0.x), but they are supported in the current nightly builds
(1.1.0-rc1 and higher).

To avoid this, this model uses strided convolutions instead of Average/Maxpooling for
downsampling. If you wish to use pooling operations in your discriminator, please ensure
you update Tensorflow to 1.1.0-rc1 or higher. I haven't tested this with Theano at all.

The model saves images using pillow. If you don't have pillow, either install it or
remove the calls to generate_images.
"""
import argparse
import os
import numpy as np
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers.merge import _Merge
from keras.layers.convolutional import Convolution2D, Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from keras.datasets import mnist
from keras import backend as K
from functools import partial

try:
    from PIL import Image
except ImportError:
    print('This script depends on pillow! '
          'Please install it (e.g. with pip install pillow)')
    exit()

BATCH_SIZE = 64
# The training ratio is the number of discriminator updates
# per generator update. The paper uses 5.
TRAINING_RATIO = 5
GRADIENT_PENALTY_WEIGHT = 10  # As per the paper


def wasserstein_loss(y_true, y_pred):
    """Calculates the Wasserstein loss for a sample batch.

    The Wasserstein loss function is very simple to calculate. In a standard GAN, the
    discriminator has a sigmoid output, representing the probability that samples are
    real or generated. In Wasserstein GANs, however, the output is linear with no
    activation function! Instead of being constrained to [0, 1], the discriminator wants
    to make the distance between its output for real and generated samples as
    large as possible.

    The most natural way to achieve this is to label generated samples -1 and real
    samples 1, instead of the 0 and 1 used in normal GANs, so that multiplying the
    outputs by the labels will give you the loss immediately.

    Note that the nature of this loss means that it can be (and frequently will be)
    less than 0."""
    return K.mean(y_true * y_pred)


def gradient_penalty_loss(y_true, y_pred, averaged_samples,
                          gradient_penalty_weight):
    """Calculates the gradient penalty loss for a batch of "averaged" samples.

    In Improved WGANs, the 1-Lipschitz constraint is enforced by adding a term to the
    loss function that penalizes the network if the gradient norm moves away from 1.
    However, it is impossible to evaluate this function at all points in the input
    space. The compromise used in the paper is to choose random points on the lines
    between real and generated samples, and check the gradients at these points. Note
    that it is the gradient w.r.t. the input averaged samples, not the weights of the
    discriminator, that we're penalizing!

    In order to evaluate the gradients, we must first run samples through the generator
    and evaluate the loss. Then we get the gradients of the discriminator w.r.t. the
    input averaged samples. The l2 norm and penalty can then be calculated for this
    gradient.

    Note that this loss function requires the original averaged samples as input, but
    Keras only supports passing y_true and y_pred to loss functions. To get around this,
    we make a partial() of the function with the averaged_samples argument, and use that
    for model training."""
    # first get the gradients:
    #   assuming: - that y_pred has dimensions (batch_size, 1)
    #             - averaged_samples has dimensions (batch_size, nbr_features)
    # gradients afterwards has dimension (batch_size, nbr_features), basically
    # a list of nbr_features-dimensional gradient vectors
    gradients = K.gradients(y_pred, averaged_samples)[0]
    # compute the euclidean norm by squaring ...
    gradients_sqr = K.square(gradients)
    #   ... summing over the rows ...
    gradients_sqr_sum = K.sum(gradients_sqr,
                              axis=np.arange(1, len(gradients_sqr.shape)))
    #   ... and sqrt
    gradient_l2_norm = K.sqrt(gradients_sqr_sum)
    # compute lambda * (1 - ||grad||)^2 still for each single sample
    gradient_penalty = gradient_penalty_weight * K.square(1 - gradient_l2_norm)
    # return the mean as loss over all the batch samples
    return K.mean(gradient_penalty)


def make_generator():
    """Creates a generator model that takes a 100-dimensional noise vector as a "seed",
    and outputs images of size 28x28x1."""
    model = Sequential()
    model.add(Dense(1024, input_dim=100))
    model.add(LeakyReLU())
    model.add(Dense(128 * 7 * 7))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    if K.image_data_format() == 'channels_first':
        model.add(Reshape((128, 7, 7), input_shape=(128 * 7 * 7,)))
        bn_axis = 1
    else:
        model.add(Reshape((7, 7, 128), input_shape=(128 * 7 * 7,)))
        bn_axis = -1
    model.add(Conv2DTranspose(128, (5, 5), strides=2, padding='same'))
    model.add(BatchNormalization(axis=bn_axis))
    model.add(LeakyReLU())
    model.add(Convolution2D(64, (5, 5), padding='same'))
    model.add(BatchNormalization(axis=bn_axis))
    model.add(LeakyReLU())
    model.add(Conv2DTranspose(64, (5, 5), strides=2, padding='same'))
    model.add(BatchNormalization(axis=bn_axis))
    model.add(LeakyReLU())
    # Because we normalized training inputs to lie in the range [-1, 1],
    # the tanh function should be used for the output of the generator to ensure
    # its output also lies in this range.
    model.add(Convolution2D(1, (5, 5), padding='same', activation='tanh'))
    return model


def make_discriminator():
    """Creates a discriminator model that takes an image as input and outputs a single
    value, representing whether the input is real or generated. Unlike normal GANs, the
    output is not sigmoid and does not represent a probability! Instead, the output
    should be as large and negative as possible for generated inputs and as large and
    positive as possible for real inputs.

    Note that the improved WGAN paper suggests that BatchNormalization should not be
    used in the discriminator."""
    model = Sequential()
    if K.image_data_format() == 'channels_first':
        model.add(Convolution2D(64, (5, 5), padding='same', input_shape=(1, 28, 28)))
    else:
        model.add(Convolution2D(64, (5, 5), padding='same', input_shape=(28, 28, 1)))
    model.add(LeakyReLU())
    model.add(Convolution2D(128, (5, 5), kernel_initializer='he_normal',
                            strides=[2, 2]))
    model.add(LeakyReLU())
    model.add(Convolution2D(128, (5, 5), kernel_initializer='he_normal', padding='same',
                            strides=[2, 2]))
    model.add(LeakyReLU())
    model.add(Flatten())
    model.add(Dense(1024, kernel_initializer='he_normal'))
    model.add(LeakyReLU())
    model.add(Dense(1, kernel_initializer='he_normal'))
    return model


def tile_images(image_stack):
    """Given a stacked tensor of images, reshapes them into a horizontal tiling for
    display."""
    assert len(image_stack.shape) == 3
    image_list = [image_stack[i, :, :] for i in range(image_stack.shape[0])]
    tiled_images = np.concatenate(image_list, axis=1)
    return tiled_images


class RandomWeightedAverage(_Merge):
    """Takes a randomly-weighted average of two tensors. In geometric terms, this
    outputs a random point on the line between each pair of input points.

    Inheriting from _Merge is a little messy but it was the quickest solution I could
    think of. Improvements appreciated."""

    def _merge_function(self, inputs):
        weights = K.random_uniform((BATCH_SIZE, 1, 1, 1))
        return (weights * inputs[0]) + ((1 - weights) * inputs[1])


def generate_images(generator_model, output_dir, epoch):
    """Feeds random seeds into the generator and tiles and saves the output to a PNG
    file."""
    test_image_stack = generator_model.predict(np.random.rand(10, 100))
    test_image_stack = (test_image_stack * 127.5) + 127.5
    test_image_stack = np.squeeze(np.round(test_image_stack).astype(np.uint8))
    tiled_output = tile_images(test_image_stack)
    tiled_output = Image.fromarray(tiled_output, mode='L')  # L specifies greyscale
    outfile = os.path.join(output_dir, 'epoch_{}.png'.format(epoch))
    tiled_output.save(outfile)


parser = argparse.ArgumentParser(description="Improved Wasserstein GAN "
                                             "implementation for Keras.")
parser.add_argument("--output_dir", "-o", required=True,
                    help="Directory to output generated files to")
args = parser.parse_args()

# First we load the image data, reshape it and normalize it to the range [-1, 1]
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = np.concatenate((X_train, X_test), axis=0)
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1], X_train.shape[2]))
else:
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
X_train = (X_train.astype(np.float32) - 127.5) / 127.5

# Now we initialize the generator and discriminator.
generator = make_generator()
discriminator = make_discriminator()

# The generator_model is used when we want to train the generator layers.
# As such, we ensure that the discriminator layers are not trainable.
# Note that once we compile this model, updating .trainable will have no effect within
# it. As such, it won't cause problems if we later set discriminator.trainable = True
# for the discriminator_model, as long as we compile the generator_model first.
for layer in discriminator.layers:
    layer.trainable = False
discriminator.trainable = False
generator_input = Input(shape=(100,))
generator_layers = generator(generator_input)
discriminator_layers_for_generator = discriminator(generator_layers)
generator_model = Model(inputs=[generator_input],
                        outputs=[discriminator_layers_for_generator])
# We use the Adam paramaters from Gulrajani et al.
generator_model.compile(optimizer=Adam(0.0001, beta_1=0.5, beta_2=0.9),
                        loss=wasserstein_loss)

# Now that the generator_model is compiled, we can make the discriminator
# layers trainable.
for layer in discriminator.layers:
    layer.trainable = True
for layer in generator.layers:
    layer.trainable = False
discriminator.trainable = True
generator.trainable = False

# The discriminator_model is more complex. It takes both real image samples and random
# noise seeds as input. The noise seed is run through the generator model to get
# generated images. Both real and generated images are then run through the
# discriminator. Although we could concatenate the real and generated images into a
# single tensor, we don't (see model compilation for why).
real_samples = Input(shape=X_train.shape[1:])
generator_input_for_discriminator = Input(shape=(100,))
generated_samples_for_discriminator = generator(generator_input_for_discriminator)
discriminator_output_from_generator = discriminator(generated_samples_for_discriminator)
discriminator_output_from_real_samples = discriminator(real_samples)

# We also need to generate weighted-averages of real and generated samples,
# to use for the gradient norm penalty.
averaged_samples = RandomWeightedAverage()([real_samples,
                                            generated_samples_for_discriminator])
# We then run these samples through the discriminator as well. Note that we never
# really use the discriminator output for these samples - we're only running them to
# get the gradient norm for the gradient penalty loss.
averaged_samples_out = discriminator(averaged_samples)

# The gradient penalty loss function requires the input averaged samples to get
# gradients. However, Keras loss functions can only have two arguments, y_true and
# y_pred. We get around this by making a partial() of the function with the averaged
# samples here.
partial_gp_loss = partial(gradient_penalty_loss,
                          averaged_samples=averaged_samples,
                          gradient_penalty_weight=GRADIENT_PENALTY_WEIGHT)
# Functions need names or Keras will throw an error
partial_gp_loss.__name__ = 'gradient_penalty'

# Keras requires that inputs and outputs have the same number of samples. This is why
# we didn't concatenate the real samples and generated samples before passing them to
# the discriminator: If we had, it would create an output with 2 * BATCH_SIZE samples,
# while the output of the "averaged" samples for gradient penalty
# would have only BATCH_SIZE samples.

# If we don't concatenate the real and generated samples, however, we get three
# outputs: One of the generated samples, one of the real samples, and one of the
# averaged samples, all of size BATCH_SIZE. This works neatly!
discriminator_model = Model(inputs=[real_samples,
                                    generator_input_for_discriminator],
                            outputs=[discriminator_output_from_real_samples,
                                     discriminator_output_from_generator,
                                     averaged_samples_out])
# We use the Adam paramaters from Gulrajani et al. We use the Wasserstein loss for both
# the real and generated samples, and the gradient penalty loss for the averaged samples
discriminator_model.compile(optimizer=Adam(0.0001, beta_1=0.5, beta_2=0.9),
                            loss=[wasserstein_loss,
                                  wasserstein_loss,
                                  partial_gp_loss])
# We make three label vectors for training. positive_y is the label vector for real
# samples, with value 1. negative_y is the label vector for generated samples, with
# value -1. The dummy_y vector is passed to the gradient_penalty loss function and
# is not used.
positive_y = np.ones((BATCH_SIZE, 1), dtype=np.float32)
negative_y = -positive_y
dummy_y = np.zeros((BATCH_SIZE, 1), dtype=np.float32)

for epoch in range(100):
    np.random.shuffle(X_train)
    print("Epoch: ", epoch)
    print("Number of batches: ", int(X_train.shape[0] // BATCH_SIZE))
    discriminator_loss = []
    generator_loss = []
    minibatches_size = BATCH_SIZE * TRAINING_RATIO
    for i in range(int(X_train.shape[0] // (BATCH_SIZE * TRAINING_RATIO))):
        discriminator_minibatches = X_train[i * minibatches_size:
                                            (i + 1) * minibatches_size]
        for j in range(TRAINING_RATIO):
            image_batch = discriminator_minibatches[j * BATCH_SIZE:
                                                    (j + 1) * BATCH_SIZE]
            noise = np.random.rand(BATCH_SIZE, 100).astype(np.float32)
            discriminator_loss.append(discriminator_model.train_on_batch(
                [image_batch, noise],
                [positive_y, negative_y, dummy_y]))
        generator_loss.append(generator_model.train_on_batch(np.random.rand(BATCH_SIZE,
                                                                            100),
                                                             positive_y))
    # Still needs some code to display losses from the generator and discriminator,
    # progress bars, etc.
    generate_images(generator, args.output_dir, epoch)